# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [58]:
previous_data = "../WeatherPy/output/city_data.csv"
weather_data = pd.read_csv(previous_data)

weather_data = weather_data.drop(index=196)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [59]:
city = weather_data[['Lat','Lng']].astype(float)
humidity = weather_data['Humidity'].astype(float)

fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(city, weights = humidity)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:

vacation_df = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])
max_temp = (vacation_df["Max Temp"] <= 80) & (vacation_df["Max Temp"] > 70)
wind_speed = vacation_df["Wind Speed"] < 10
cloudiness = vacation_df["Cloudiness"] == 0
vacation_df = vacation_df[max_temp & wind_speed & cloudiness]

vacation_df

,City,Max Temp,Wind Speed,Cloudiness
18,changji,78.80,8.95,0
60,sorong,78.19,3.67,0
93,singaraja,76.19,3.06,0
167,manono,72.72,5.88,0
223,jiutai,73.40,6.71,0
256,luderitz,71.60,4.70,0
291,meligalas,79.00,8.05,0
357,zaysan,71.92,7.00,0
366,krasnoyarskiy,73.04,8.70,0
375,abalak,72.03,4.61,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""


In [63]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotels = []
count = 0



for i in range(len(hotel_df)):
    #count +=1
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']
    #print(i)
    #print(hotel_df.loc[i]['Lat'])
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    request = requested.json()
    try:
        hotels.append(request['results'][0]['name'])
    except:
        hotels.append("")



hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

KeyError: 196

In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))